In [6]:
import numpy as np
import nlopt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Load data
X = np.load('data/X.npy')
y = np.load('data/y.npy')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

# Define objective function to be minimized (negative accuracy)
def objective(params, X, y, model_func):
    # Set kernel
    kernel = 'rbf'

    # Set C and gamma
    C = params[0]
    gamma = params[1]

    # Train SVM model with given hyperparameters
    model = model_func(X, y, C=C, gamma=gamma, kernel=kernel)

    # Evaluate accuracy on validation set
    acc = model.score(X_val, y_val)

    # Return negative accuracy for minimization
    return -acc

# Define constraints for hyperparameters
def constraint(params):
    C = params[0]
    gamma = params[1]
    return C - 1e-6, gamma - 1e-6

# Define function to optimize SVM hyperparameters with newuoa
def optimize_svm(X, y, model_func, bounds):
    # Set initial guess for hyperparameters
    init_guess = [1.0, 1.0]

    # Define optimization problem
    opt = nlopt.opt(nlopt.LN_NEWUOA, 2)
    opt.set_min_objective(lambda params: objective(params, X, y, model_func))
    opt.set_lower_bounds([bounds[0][0], bounds[1][0]])
    opt.set_upper_bounds([bounds[0][1], bounds[1][1]])
    opt.add_equality_constraint(constraint)

    # Run optimization
    params = opt.optimize(init_guess)

    # Train SVM model with optimized hyperparameters
    C_opt = params[0]
    gamma_opt = params[1]
    model_opt = model_func(X, y, C=C_opt, gamma=gamma_opt, kernel='rbf')

    # Print best hyperparameters and accuracy on validation set
    print(f"Best hyperparameters: C = {C_opt:.6f}, gamma = {gamma_opt:.6f}")
    print(f"Accuracy on validation set: {model_opt.score(X_val, y_val):.6f}")

    return C_opt, gamma_opt, model_opt

# Define bounds for hyperparameters
bounds = [(1e-5, 10), (1e-5, 10)]

# Call optimization function with SVM as model function
C_opt, gamma_opt, model_opt = optimize_svm(X_train, y_train, SVC, bounds)


FileNotFoundError: [Errno 2] No such file or directory: 'data/X.npy'